In [2]:
import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Paths
input_path = r'data'
output_path = r'result\data_result'
os.makedirs(output_path, exist_ok=True)

# Classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
object_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')  # General object detection on face

# Function to classify based on filename
def classify_face(filename):
    return 'real' if filename.startswith('P') else 'not real'

# Function for Non-Maximum Suppression (NMS)
def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2] + boxes[:,0]
    y2 = boxes[:,3] + boxes[:,1]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

# Initialize metrics
TP = TN = FP = FN = 0
y_true_faces = []
y_pred_faces = []
y_true_objects = []
y_pred_objects = []

# Process each image
for filename in os.listdir(input_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img = cv2.imread(os.path.join(input_path, filename))
        
        # Convert to grayscale and apply CLAHE to each color channel
        channels = cv2.split(img)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        clahe_channels = [clahe.apply(channel) for channel in channels]
        clahe_image = cv2.merge(clahe_channels)
        
        gray_image = cv2.cvtColor(clahe_image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0)  # Reduced blur for sharper edges
        
        # Face detection with refined parameters
        faces = face_cascade.detectMultiScale(
            gray_image, scaleFactor=1.03, minNeighbors=8, minSize=(30, 30)
        )
        faces = non_max_suppression(faces, 0.3)
        classification = classify_face(filename)
        
        if len(faces) == 0:
            TN += (classification == 'not real')
            FN += (classification == 'real')
            y_true_faces.append(0)  # 0 = not real
            y_pred_faces.append(0)
        else:
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), 2)
                cv2.putText(img, classification, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                
            TP += (classification == 'real')
            FP += (classification == 'not real')
            y_true_faces.append(1)  # 1 = real
            y_pred_faces.append(1)
            
            # Restrict object detection to the face region
            for (x, y, w, h) in faces:
                face_region = gray_image[y:y+h, x:x+w]  # Crop the face region from the grayscale image
                objects = object_cascade.detectMultiScale(
                    face_region, scaleFactor=1.03, minNeighbors=6, minSize=(15, 15)
                )
                objects = non_max_suppression(objects, 0.3)
                
                y_true_objects.append(1)  # Assume objects are present if detected
                
                if len(objects) > 0:
                    y_pred_objects.append(1)
                    for (ox, oy, ow, oh) in objects:
                        cv2.rectangle(img, (x+ox, y+oy), (x+ox+ow, y+oy+oh), (255, 255, 0), 2)
                        cv2.putText(img, 'Object', (x+ox, y+oy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                else:
                    y_pred_objects.append(0)
        
        # Save the processed image
        cv2.imwrite(os.path.join(output_path, filename), img)
        print(f'Saved {filename} with classification {classification}')

# Calculate performance metrics
total_images = TP + TN + FP + FN
print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
print(f"Accuracy: {(TP + TN) / total_images * 100:.2f}%")

# Confusion Matrix and Classification Report for Face Detection
conf_matrix_faces = confusion_matrix(y_true_faces, y_pred_faces)
print("Confusion Matrix for Faces:\n", conf_matrix_faces)
print("Classification Report for Faces:\n", classification_report(y_true_faces, y_pred_faces))

# Confusion Matrix and Classification Report for Object Detection
conf_matrix_objects = confusion_matrix(y_true_objects, y_pred_objects)
print("Confusion Matrix for Objects:\n", conf_matrix_objects)
print("Classification Report for Objects:\n", classification_report(y_true_objects, y_pred_objects))


Saved N_01.jpg with classification not real
Saved N_02.jpg with classification not real
Saved N_03.jpg with classification not real
Saved N_04.jpg with classification not real
Saved N_05.jpg with classification not real
Saved N_06.jpg with classification not real
Saved N_07.jpg with classification not real
Saved N_08.jpg with classification not real
Saved N_09.jpg with classification not real
Saved N_10.jpg with classification not real
Saved N_11.jpg with classification not real
Saved N_12.jpg with classification not real
Saved N_13.jpg with classification not real
Saved N_14.jpg with classification not real
Saved N_15.jpg with classification not real
Saved N_16.jpg with classification not real
Saved N_17.jpg with classification not real
Saved N_18.jpg with classification not real
Saved N_19.jpg with classification not real
Saved N_20.jpg with classification not real
Saved N_21.jpg with classification not real
Saved N_22.jpg with classification not real
Saved N_23.jpg with classificati

c:\Users\seahc\anaconda3\envs\cve\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\seahc\anaconda3\envs\cve\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\seahc\anaconda3\envs\cve\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
